In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin, urlparse, parse_qs
import datetime
from tqdm import tqdm_notebook

#### 1. 뉴스 속보 페이지에서 정치/경제/사회 기사 링크 수집 
링크 : https://news.naver.com/main/ranking/popularDay.nhn

In [2]:
# 수집해야할 날짜의 범위
def news_link_crawler(date_range, base_url, params):

    ranking_news_list = []
    
    for date in date_range:
        params['date'] = date

        resp = requests.get(base_url,params=params)
        soup = BeautifulSoup(resp.text)
        ranking_wrap = soup.select('div.ranking_section li dt a')

        date_news_dict = {}
        news_list = []

        # 정치, 경제, 사회
        for ranking in ranking_wrap[:-15]:
            news_list.append(urljoin(base_url,ranking.get('href')))

        date_news_dict[date] = news_list

        ranking_news_list.append(date_news_dict)
    
    return ranking_news_list


#### 뉴스 기사에서 기사제목/ 섹션 / 요약봇의 기사요약 내용 / 이미지 추출

In [3]:
def news_body_crawler(ranking_news_list):
    finally_news_list = []

    for ranking_list in tqdm_notebook(ranking_news_list):
        for yyyymmdd, ranking in ranking_list.items():
            for ranking_news in ranking:

                news_dict = {}
                
                resp_news = requests.get(ranking_news)
                soup_news = BeautifulSoup(resp_news.text)

                # 기사 제목
                header_wrap = soup_news.select('div.article_info h3')
                title = header_wrap[0].text

                # 섹션
                section = soup_news.find('em', class_='guide_categorization_item')

                # 기사 ID
                parsed_url = urlparse(ranking_news)
                qs = parse_qs(parsed_url.query)
                oid = qs['oid'][0]
                aid = qs['aid'][0]
                # print(oid, aid)

                # 내용
                summary_url = 'https://tts.news.naver.com/article/{}/{}/summary'.format(oid, aid)
                resp_summary = requests.get(summary_url)
                soup_summary = BeautifulSoup(resp_summary.text)
                summary = soup_summary.text.split(':')[-1].replace('}','').replace("\\","")
                if summary:
                    pass
                else:
                    summary = '요약 제공 안함'
                    
                news_dict = {
                    '날짜' : yyyymmdd[:4]+'-'+yyyymmdd[4:6]+'-'+yyyymmdd[6:],
                    'title' : title.replace(',',''),
                    'category' : section.text,
                    'summary' : summary.replace('"','').replace(',','')
                }
                finally_news_list.append(news_dict)
    return finally_news_list

In [4]:
# def save_df_to_csv(finally_news_list):
#     df = pd.DataFrame(finally_news_list)
#     df.to_csv('naver_news_crawling.csv', encoding='utf-8', index=False)

In [5]:
date_range = (pd.date_range(start='20200501', end=datetime.date.today())).strftime("%Y%m%d").tolist()
base_url= 'https://news.naver.com/main/ranking/popularDay.nhn'
params = {
    'rankingType' : 'popular_day',
    'date' : ''
}

NewsList = news_link_crawler(date_range, base_url, params)
FinallyNewsList = news_body_crawler(NewsList)
df_news = pd.DataFrame(FinallyNewsList)
print('=====END News Crawling=====')

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))



=====END News Crawling=====


In [6]:
df_news

,날짜,title,category,summary
0,2020-05-01,지성호 “김정은 지난 주말 수술 쇼크로 사망했다”,정치,탈북민 출신 인권운동가 지성호 미래한국당 국회의원 당선인이 김정은 북한 국무위원장의...
1,2020-05-01,"지성호 ""김정은 지난 주말 사망.. 99% 확신"" 주장",정치,탈북자 출신 인권운동가 지성호 미래한국당 비례대표 당선자는 1일 김정은이 사망한 것...
2,2020-05-01,"[인터뷰] ""김정은 사망 99% 확신"" 지성호 주장 이유는?",정치,오늘 우리 지성호 당선인 또 미래통합당의 태구민 태영호 당선인이 대한민국 국회의원...
3,2020-05-01,"휴가 취소하고 걸어서 곰탕집 간 文 ""식당 이용 활발했으면""",정치,문재인 대통령이 1일 참모진들과 청와대 주변 한 곰탕집을 찾아 오찬을 했다.문 대통...
4,2020-05-01,"지성호 ""김정은 사망 99% 확신"" 청와대 ""특이동향 없다""(종합)",정치,탈북자 출신으로 4·15 총선에서 미래한국당 비례대표로 당선된 지성호 당선인은 1일...
...,...,...,...,...
820,2020-06-24,[속보] 충남 부여 50대女 코로나 확진…“대전 확진자 접촉”,사회,충남 부여군은 24일 부여읍에 거주하는 50대 여성이 코로나19 확진 판정을 받았다...
821,2020-06-24,"[속보] 충남 부여서 50대 여성 확진…""대전 93번 확진자 접촉""",사회,해당 기사는 요청하신 자동 요약문을 제공할 수 없습니다.
822,2020-06-24,쿠팡 물류센터 '또' 확진…2천 명 근무하는데,사회,그런가 하면 쿠팡 물류 센터에서또 확진자가 발생했습니다.경기도 이천시 덕평 물류 센...
823,2020-06-24,"신동주 故신격호 유언장 발견장소에 의문 ""생전 발언과 달라""",사회,롯데그룹 창업주인 고 신격호 명예회장의 자필 유언장이 24일 공개된 가운데 장남인 ...
